# Visualize HDF5 in a knowledge graph

Using `kglab` (https://derwen.ai/docs/kgl/)

In [1]:
# !pip install kglab

In [2]:
import kglab
import rdflib
import numpy as np

In [3]:
import h5rdmtoolbox as h5tbx

In [4]:
with h5tbx.File() as h5:
    h5.create_dataset(name='test', data=4.3)
    h5.create_dataset(name='nd-array', data=np.random.rand(2, 3), chunks=(1,3))
    grp = h5.create_group(name='contact')
    grp.attrs['fname', rdflib.FOAF.firstName] = 'Matthias'
    grp.attrs['lname', rdflib.FOAF.lastName] = 'Probst'
    grp.attrs['hint'] = 'This group could be representing a person.'
    grp.rdf['hint'].definition = 'A hint gives helpful information on something.'
    grp.rdf.type = rdflib.PROV.Person  # --> rdf.type = Person
    grp.rdf.subject = 'https://orcid.org/0000-0001-8729-0482'  # -> @id='https://orcid.org/0000-0001-8729-0482'
    
graph = h5tbx.get_ld(h5.hdf_filename, structural=True, file_uri="https://example.org#", context={"ex": "https://example.org#"})
print(graph.serialize())

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix ex: <https://example.org#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix hdf: <http://purl.allotrope.org/ontologies/hdf5/1.8#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:tmp0.hdf a hdf:File ;
    hdf:rootGroup <https://example.org#tmp0.hdf/> .

<https://example.org#tmp0.hdf/> a hdf:Group ;
    hdf:member <https://example.org#tmp0.hdf/contact>,
        <https://example.org#tmp0.hdf/nd-array>,
        <https://example.org#tmp0.hdf/test> ;
    hdf:name "/"^^xsd:string .

<https://example.org#tmp0.hdf/contact> a hdf:Group ;
    hdf:attribute <https://example.org#tmp0.hdf/contact@fname>,
        <https://example.org#tmp0.hdf/contact@hint>,
        <https://example.org#tmp0.hdf/contact@lname> ;
    hdf:name "/contact"^^xsd:string ;
    dcterms:relation <https://orcid.org/0000-0001-8729-0482> .

<https://example.org#tmp0.hdf/contact@fname> a hdf:StringAttribute ;
    hdf:

In [5]:
namespaces = {k: str(v) for k, v in dict(graph.namespaces()).items()}
kg = kglab.KnowledgeGraph(
    import_graph=graph,
    namespaces=namespaces)

In [6]:
subgraph = kglab.SubgraphTensor(kg)

In [7]:
VIS_STYLE= {
    "hdf": {"color": "red", "size": 30},
    "_": {"color": "black", "size": 20}
}

In [8]:
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)
# pyvis_graph.show('graph.html', notebook=True)
# the above call fails in the readthedocs build process... we show an image of the result here:

![grafik.png](../_static/Screenshot_pyvis_graph.png)

In [9]:
with h5tbx.File() as h5:
    h5.create_dataset(name='test', data=4.3)
    grp = h5.create_group(name='contact')
    grp.attrs['fname', rdflib.FOAF.firstName] = 'Matthias'
    grp.attrs['lname', rdflib.FOAF.lastName] = 'Probst'
    grp.attrs['hint'] = 'bah blah'
    grp.rdf['hint'].definition = 'A hint'
    grp.rdf.subject = rdflib.PROV.Person
    grp.attrs['@id'] = 'https://orcid.org/0000-0001-8729-0482'
    print(h5.dump_jsonld(indent=2, structural=False, file_uri="https://example.org#"))

graph = h5tbx.get_ld(h5.hdf_filename, structural=False, file_uri="https://example.org#")

{
  "@context": {
    "dcterms": "http://purl.org/dc/terms/",
    "foaf": "http://xmlns.com/foaf/0.1/",
    "prov": "http://www.w3.org/ns/prov#"
  },
  "@graph": [
    {
      "@id": "prov:Person",
      "foaf:firstName": "Matthias",
      "foaf:lastName": "Probst"
    },
    {
      "@id": "https://example.org#tmp1.hdf/contact",
      "dcterms:relation": {
        "@id": "prov:Person"
      }
    }
  ]
}


In [10]:
VIS_STYLE= {
    "hdf": {"color": "red", "size": 30},
    "_": {"color": "black", "size": 20},
    "prof": {"color": "green", "size": 10},
    "foaf": {"color": "green", "size": 10}
}

In [11]:
kg_from_jsonld = kglab.KnowledgeGraph().load_rdf_text(
    graph.serialize(format="ttl")
)

subgraph = kglab.SubgraphTensor(kg_from_jsonld)
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)

pyvis_graph.force_atlas_2based(overlap=0)
# pyvis_graph.show("tmp.grpah.html", local=True, notebook=True)
# the above call fails in the readthedocs build process... we show an image of the result here:

![grafik.png](../_static/Screenshot_pyvis_graph2.png)